## Continue Pre-training Kinyarwanda 


this is an experimental notebook to fine-tune llama 3 for Kinyarwanda 

in this notebook we only try the "continous pre-training"

(we leave it to later work the "instruction-finetuning"). 


we use 
- llama2-8b as basis model (a 4bit quantized version) 
- Unsloth as a fine-tuning framework 
- datasets: kinyarwanda - wikipedia & kinyarwanda news (see notebook on dataset about their preparation) 




In [ ]:
#Import libraries 

from unsloth import FastLanguageModel
import torch

from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
from unsloth import UnslothTrainer, UnslothTrainingArguments


from datasets import load_dataset
from datasets import Dataset

import json 
import pandas as pd 




## 1 . loading the model & fine-tuning parameters 

In [ ]:
# we use unsloth & here we load the model 

max_seq_length = 2048 # this can be adapted for longer context 
dtype = None # the datatype will be auto-detected : Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # we use 4bit quantization to reduce memory usage. 


xmodel = 'unsloth/llama-3-8b-bnb-4bit'

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = xmodel , 
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)



In [ ]:
## parameters 

model = FastLanguageModel.get_peft_model(
    model,
    r = 128, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",

                      "embed_tokens", "lm_head",], # Add for continual pretraining
    lora_alpha = 32,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = True,   # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

## 2 . loading the dataset 

see notebook **Kinyarwanda_Finetuning_Datasets** on how the datasets were created 


In [ ]:
xfiles = ['kinyarwanda_monolingual_rwandannews.jsonl',
          'kinyarwanda_monolingual_wikipedia20231101.jsonl']

text_data = []

for xfile in xfiles:
    xfile_name = './_datasets/'+ xfile
    with open(xfile_name, 'r') as file:
        for line in file:
            xjson = json.loads(line)
            if xfile == 'kinyarwanda_monolingual_wikipedia20231101.jsonl':
                xtext_field = xjson.get('title') + ' ' + xjson.get('text')
            else:
                xtext_field =  xjson.get('text')                
                
            xdict_text = {'text': xtext_field}
            
            
            text_data.append(xdict_text)

# into a dataset 
dataset = Dataset.from_pandas(pd.DataFrame(text_data))

# shuffle 

dataset = dataset.shuffle(seed=42)

print(len(dataset)) 
    

In [ ]:
dataset[1]

## 3  Training arguments 

In [ ]:
trainer = UnslothTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,

    args = UnslothTrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 8,

        # Use warmup_ratio and num_train_epochs for longer runs!
        #max_steps = 120,
        #warmup_steps = 10,
        warmup_ratio = 0.1,
        num_train_epochs = 2, 

        # Select a 2 to 10x smaller learning rate for the embedding matrices!
        learning_rate = 5e-5,
        embedding_learning_rate = 1e-5,

        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs"
    ))

In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

## train 

In [ ]:
trainer_stats = trainer.train()

In [ ]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

# inference 

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer('Imana ', return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 1000, use_cache = True)
tokenizer.batch_decode(outputs)

In [ ]:
xprompt = 'Umugabo yaraje abwira abantu ati '


# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(xprompt, return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 500, use_cache = False)
q1 = tokenizer.batch_decode(outputs)

print(q1[0])


In [ ]:
# alpaca_prompt = Copied from above

xprompt = '''mu gihugu cy'ubufaransa'''

FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(xprompt, return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 500, use_cache = False)
q1 = tokenizer.batch_decode(outputs)
print(q1[0])

In [ ]:
xprompt = '''amateka ya Afurika'''

FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(xprompt, return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 500, use_cache = False)
q1 = tokenizer.batch_decode(outputs)

print(q1[0])

In [ ]:
xprompt = '''Ejo bundi umugabo yaje nk'iya Gatera '''

FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(xprompt, return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 500, use_cache = False)
q1 = tokenizer.batch_decode(outputs)
print(q1[0])

In [ ]:
xprompt = '''the history of the persian empire  '''

FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(xprompt, return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 500, use_cache = False)
q1 = tokenizer.batch_decode(outputs)

print(q1[0])

In [ ]:
xprompt = '''umwana wange yarambwiye   '''
xprompt = '''Ejo bundi umwana yagiye '''


FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(xprompt, return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 256, use_cache = True)
q1 = tokenizer.batch_decode(outputs)

print(q1[0])

## save model 



In [ ]:
model.save_pretrained("llamarwanda_rw_v1") # Local saving
tokenizer.save_pretrained("llamarwanda_rw_v1")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

## SAVE FULL MODEL 

In [ ]:
max_seq_length = 2048 # this can be adapted for longer context 
dtype = None # the datatype will be auto-detected : Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # we use 4bit quantization to reduce memory usage. 


xmodel = 'unsloth/llama-3-8b-bnb-4bit'

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = xmodel , 
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

In [ ]:

max_seq_length = 2048 # this can be adapted for longer context 
dtype = None # the datatype will be auto-detected : Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # we use 4bit quantization to reduce memory usage. 


xmodel = '/home/mike/xGitHubRepos/kinyarwanda_ft_llm/02_continue_pretraining/llamarwanda_rw_v002'

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = xmodel , 
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

In [ ]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

#### test before testing 

In [ ]:
def complete_prompt(xprompt):
    inputs = tokenizer(xprompt, return_tensors = "pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens = 1000, use_cache = True)
    q1 = tokenizer.batch_decode(outputs)
    
    return q1
    

In [ ]:
xresp = complete_prompt("Umugaba w'ingabo")[0]

xresp

In [ ]:
model.save_pretrained_merged("llamarwanda_full_v002", 
                             tokenizer, 
                             save_method = "merged_16bit",)




## save to hub 

### !! NOT DONE . we will save the second iteration of the model (more data, trained for more epoch and longer context size) 



In [ ]:
import sys
import yaml

#xConfigFile = 'C:/Users/mugabal/___MyFiles/xconfig.yaml'
xConfigFile = '/home/mike/_____ConfigParameters/xconfig.yaml'
with open(xConfigFile, 'r') as xstream:
	xConfigParams = yaml.safe_load(xstream)
	sys.path.append(xConfigParams['xpath_scientopy'])


xKey = xConfigParams.get('huggingface').get('api_write')

In [ ]:
model.push_to_hub_merged("almugabo/kinyallm_base_llama3_v0", 
                         tokenizer, 
                         save_method = "merged_16bit", token = xKey)

print('OK')